In [5]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from time import time
import pandas as pd
from  os.path import join
import pickle as pkl
data_path = './data/'
output_path = './output/'

In [ ]:
data = pkl.load(open(join(data_path,'good-good_30G.pkl'), 'rb'))
# Step 1: 转换为稀疏布尔矩阵
te = TransactionEncoder()
te_result = te.fit(data).transform(data, sparse=True)

# 使用稀疏DataFrame节省内存
df = pd.DataFrame.sparse.from_spmatrix(te_result, columns=te.columns_)

# Step 2: 执行 FP-Growth，设置最小支持度 ≥ 0.02
start_time = time()
frequent_itemsets = fpgrowth(df, min_support=0.02, use_colnames=True)
print(f"频繁项集数量: {len(frequent_itemsets)}")
print(f"耗时: {time() - start_time:.2f} 秒")


/tmp/ipykernel_2204334/991384046.py:6: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df = pd.DataFrame.sparse.from_spmatrix(te_result, columns=te.columns_)


In [ ]:

# out = pd.DataFrame()
# out['support'] = frequent_itemsets['support']
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ','.join(list(x)))
# frequent_itemsets['itemsets'] =  frequent_itemsets['itemsets'].apply(lambda x: x.replace(', ',''))

In [ ]:
# 将antecedents和consequents转为字符串方便写入文本
# frequent_itemsets['support'] = frequent_itemsets['support']
# frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ','.join(list(x)))

# 保存为 txt 文件，使用制表符或其他分隔符
frequent_itemsets.to_csv(join(output_path,'frequent_30G.csv'), 
                      columns=['support', 'itemsets'],
                      sep=',',  # 可以改成 ',' 以便导入 Excel
                      index=False,
                      encoding='utf-8')
# frequent_itemsets['itemsets'] 
frequent_itemsets['itemsets'] =    frequent_itemsets['itemsets'].apply(lambda x: frozenset(x.split(',')))


In [6]:

frequent_data = pd.read_csv(join(output_path,'frequent_30G.csv'))
frequent_data['itemsets'] = frequent_data['itemsets'].apply(lambda x: frozenset(x.split(',')))
frequent_itemsets = frequent_data

In [7]:

# Step 3: 生成关联规则，设置最小置信度 ≥ 0.45
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.45)
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(电子产品),(服装),0.484235,0.487229,0.222437,0.459358,0.942798,1.0,-0.013496,0.948449,-0.105254,0.296969,-0.054353,0.457947
1,(服装),(电子产品),0.487229,0.484235,0.222437,0.456536,0.942798,1.0,-0.013496,0.949032,-0.105804,0.296969,-0.053705,0.457947
2,(食品),(电子产品),0.484622,0.484235,0.221271,0.456584,0.942897,1.0,-0.013400,0.949116,-0.105153,0.295980,-0.053612,0.456766
3,(电子产品),(食品),0.484235,0.484622,0.221271,0.456949,0.942897,1.0,-0.013400,0.949041,-0.105082,0.295980,-0.053696,0.456766
4,(食品),(服装),0.484622,0.487229,0.222604,0.459335,0.942751,1.0,-0.013518,0.948409,-0.105408,0.297104,-0.054397,0.458107
5,(服装),(食品),0.487229,0.484622,0.222604,0.456878,0.942751,1.0,-0.013518,0.948917,-0.105887,0.297104,-0.053833,0.458107
6,(家居),(食品),0.251173,0.484622,0.113617,0.452347,0.933401,1.0,-0.008107,0.941066,-0.086994,0.182612,-0.062624,0.343396
7,(家居),(电子产品),0.251173,0.484235,0.113479,0.451795,0.933008,1.0,-0.008148,0.940825,-0.087497,0.182462,-0.062897,0.343071
8,(家居),(服装),0.251173,0.487229,0.114193,0.454640,0.933114,1.0,-0.008185,0.940244,-0.087361,0.182941,-0.063554,0.344506
9,(玩具),(服装),0.200289,0.487229,0.090889,0.453790,0.931369,1.0,-0.006697,0.938780,-0.084370,0.152338,-0.065212,0.320166


In [8]:

# Step 4: 可选 - 筛选支持度也符合要求的规则
rules_filtered = rules[
    (rules['support'] >= 0.02) & 
    (rules['confidence'] >= 0.45)
]


In [9]:
# 将结果保存为txt文件（以制表符分隔）
# 将antecedents和consequents转为字符串方便写入文本
rules_filtered['antecedents'] = rules_filtered['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_filtered['consequents'] = rules_filtered['consequents'].apply(lambda x: ', '.join(list(x)))

# 保存为 txt 文件，使用制表符或其他分隔符
rules_filtered.to_csv(join(output_path,'association_rules_30G.txt'), 
                      columns=['antecedents', 'consequents', 'support', 'confidence', 'lift'],
                      sep='\t',  # 可以改成 ',' 以便导入 Excel
                      index=False,
                      encoding='utf-8')
print('done')



done
